In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/content/fake_or_real_news.csv')

In [3]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
df['label'].value_counts()

,count
label,
REAL,3171
FAKE,3164


In [7]:
# prompt: Using dataframe df: i want lebel encoding on df.label

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform the 'label' column
df['label'] = le.fit_transform(df['label'])

# Display the first few rows to show the changes
# print(df.head())
df.head()
df.drop('Unnamed: 0',axis=1,inplace=True)

In [8]:
df.isnull().sum()

,0
title,0
text,0
label,0


In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
X.shape

(6335, 2)

In [12]:
y.shape


(6335,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.18.0'

In [29]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [16]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
messages=X.copy()

In [18]:
### Vocabulary size
voc_size=10000

In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus

['smell hillari fear',
 'watch exact moment paul ryan commit polit suicid trump ralli video',
 'kerri go pari gestur sympathi',
 'berni support twitter erupt anger dnc tri warn',
 'battl new york primari matter',
 'tehran usa',
 'girl horrifi watch boyfriend left facetim',
 'britain schindler die',
 'fact check trump clinton command chief forum',
 'iran reportedli make new push uranium concess nuclear talk',
 'three clinton iowa glimps fire elud hillari clinton campaign',
 'donald trump shockingli weak deleg game somehow got even wors',
 'strong solar storm tech risk today news oct video',
 'way america prepar world war',
 'trump take cruz lightli',
 'women lead differ',
 'shock michel obama hillari caught glamor date rape promot',
 'hillari clinton huge troubl america notic sick thing hidden pictur liberti writer news',
 'iran bill obama like',
 'chart explain everyth need know partisanship america',
 'slipperi slope trump propos ban muslim',
 'episod sunday wire hail deplor special g

In [21]:
messages

,title,text
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello..."
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...
...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene..."


In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1460, 3264, 8721],
 [7671, 133, 3607, 4103, 419, 6309, 8105, 9727, 5509, 771, 4348],
 [5806, 2353, 3203, 9503, 7730],
 [8561, 3690, 7302, 6279, 6549, 148, 6770, 4111],
 [3796, 7376, 6451, 5822, 1296],
 [716, 2638],
 [9187, 5480, 7671, 2761, 3355, 9971],
 [8481, 8116, 7210],
 [5963, 1521, 5509, 1619, 1536, 4724, 1712],
 [8676, 3394, 6612, 7376, 4161, 1370, 9811, 4735, 6673],
 [3735, 1619, 4635, 1846, 1145, 2925, 3264, 1619, 2935],
 [7879, 5509, 244, 2290, 2228, 5797, 2960, 2634, 2286, 4565],
 [4565, 2723, 3382, 8792, 8266, 6579, 4362, 717, 4348],
 [3781, 7769, 763, 9137, 142],
 [5509, 8883, 3892, 5781],
 [2601, 5363, 3918],
 [3159, 6300, 5444, 3264, 6336, 6639, 7485, 4803, 6036],
 [3264, 1619, 7914, 2631, 7769, 3578, 1384, 6991, 929, 2918, 2416, 7436, 4362],
 [8676, 6345, 5444, 9890],
 [5682, 3282, 6268, 8660, 9240, 4136, 7769],
 [1688, 653, 5509, 6050, 3132, 2517],
 [1824, 6544, 2697, 1877, 7209, 2430, 6720, 5160, 7076],
 [3264, 1619, 6612, 5540, 3937, 4504, 7784],
 [7376, 3134, 2143

In [23]:
onehot_repr[1]

[7671, 133, 3607, 4103, 419, 6309, 8105, 9727, 5509, 771, 4348]

In [24]:
messages.title[1]

'Watch The Exact Moment Paul Ryan Committed Political Suicide At A Trump Rally (VIDEO)'

In [25]:
sent_length=25
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1460 3264 8721]
 [   0    0    0 ... 5509  771 4348]
 [   0    0    0 ... 3203 9503 7730]
 ...
 [   0    0    0 ... 6930 2965 9649]
 [   0    0    0 ... 7809 5934 1031]
 [   0    0    0 ... 2488 5509 1296]]


In [26]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, 7671,  133, 3607, 4103,  419, 6309, 8105, 9727,
       5509,  771, 4348], dtype=int32)

In [27]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       1460, 3264, 8721], dtype=int32)

In [30]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [31]:
# ## Creating model
# embedding_vector_features=50 ##features representation
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(LSTM(100))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# print(model.summary())

In [32]:
len(embedded_docs),y.shape

(6335, (6335,))

In [33]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [62]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.5556 - loss: 0.6842 - val_accuracy: 0.7135 - val_loss: 0.5492
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 0.7961 - loss: 0.4495 - val_accuracy: 0.7905 - val_loss: 0.4561
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.9001 - loss: 0.2626 - val_accuracy: 0.7896 - val_loss: 0.5062
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.9352 - loss: 0.1805 - val_accuracy: 0.7829 - val_loss: 0.5508
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.9489 - loss: 0.1411 - val_accuracy: 0.7580 - val_loss: 0.7545
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.9584 - loss: 0.1125 - val_accuracy: 0.7666 - val_loss: 0.7838
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.9717 - loss: 0.0805 - val_accuracy: 0.7681 - val_loss: 0.8403
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - accuracy: 0.9777 - loss: 0.0695 - val_accuracy: 0.7652 - v

In [63]:
y_final

array([0, 0, 1, ..., 0, 1, 1])

In [64]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=50
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [65]:
y_pred=model.predict(X_test)

66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [66]:
y_test[0:15]

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0])

In [67]:
y_pred[6]

array([0.9995581], dtype=float32)

In [68]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [69]:
from sklearn.metrics import confusion_matrix

In [70]:
confusion_matrix(y_test,y_pred)

array([[795, 276],
       [239, 781]])

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7537063605930177